# PowerPoint Auto Generator

### python-pptx 설치

In [4]:
%pip install langchain langchain-openai langchain-community==0.0.19 python-pptx

  Using cached langchain_community-0.0.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.1.7-py3-none-any.whl.metadata (13 kB)
  Using cached langchain-0.1.6-py3-none-any.whl.metadata (13 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.23-py3-none-any.whl.metadata (6.0 kB)
  Using cached langsmith-0.0.87-py3-none-any.whl.metadata (10 kB)
Using cached langchain_community-0.0.19-py3-none-any.whl (1.6 MB)
Using cached langchain-0.1.6-py3-none-any.whl (811 kB)
Using cached langchain_core-0.1.23-py3-none-any.whl (241 kB)
Using cached langsmith-0.0.87-py3-none-any.whl (55 kB)
  Attempting uninstall: langsmith
    Found existing i

### 라이브러리 임포트

In [5]:
import os
import json
import requests
from pptx import Presentation
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

### API Key 설정

In [19]:
# os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
# os.environ["LANGCHAIN_API_KEY"] = "<YOUR-LANGCHAIN-API-KEY>"

### ChatModel 객체 생성

In [10]:
chat_model = ChatOpenAI()

### 사용자 정의 함수

In [11]:
def parse_response(response):
    slides = response.split('\n\n')
    slides_content = []
    
    for slide in slides:
        lines = slide.split('\n')
        title_line = lines[0]
        if ': ' in title_line:
            title = title_line.split(': ', 1)[1]  # Extract the title after 'Slide X: '
        else:
            title = title_line
            
        content_lines = [line.replace('- ', '') for line in lines[1:] if line != 'Content:']  # Skip line if it is 'Content:'
        content = []
        for sub_content in content_lines:
            content.append({sub_content: sub_content})
        
        slides_content.append({'title': title, 'content': content})
    return slides_content

In [12]:
def parse_content(assistant_response):
    start_index = assistant_response.find('Slide 1')
    end_index = assistant_response.find('EndOfDocument')
    assistant_response = assistant_response[start_index:end_index]
    slides_content = parse_response(assistant_response.strip())
    slides_content = slides_content
    return slides_content

In [13]:
def completion(doc_type, work_type, content, title):
    system_template="너는 파워포인트로 {doc_type} 내용을 작성하는 assistant 이고, {work_type} 에 전문성이 높아."
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_template="네가 작성해준 내용은 {content} 이야. [{title}] 에 대한 내용을 100자 내외로 문단을 구분해서 명료하게 작성해줘"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    request = chat_prompt.format_prompt(doc_type=doc_type, work_type=work_type, content=content, title=title).to_messages()
    result = chat_model(request)
    return result.content

In [14]:
def make_content(slides_content):
    for slide in slides_content:
        for content in slide['content']:
            for key in content.keys():
                response = completion(doc_type, work_type, assistant_response, key) 
                content[key] = response
                print(f"Make Content : {slide['title']} - {key} -{content}")
    return slides_content

### PPT  내용 생성

In [20]:
doc_type = "제안서"
work_type = "아키텍처"
number_of_slide = "10"
presentation_title = '데이터 통합 플랫폼 구축'
presenter_name = 'Danny'

system_template="""너는 파워포인트로 {doc_type} 내용을 작성하는 assistant 이고, {work_type} 에 전문성이 높아.
답변 형식은 Slide X(the number of the slide): (title of the content) /n Content: /n content 이야.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template="""{project} 제안서를 파워포인트로 만들고자 해.
파워포인트 슬라이드 장수는 {number_of_slide} 이야.
제안서에 들어갈 컨텐츠를 알려줘.
파워포인트 각 슬라이드의 제목은 고유해야 해.
답변 마지막에는 EndOfDocument 를 표시해줘.
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(doc_type=doc_type, work_type=work_type, project=presentation_title, number_of_slide=number_of_slide).to_messages()
result = chat_model(request)
assistant_response = result.content
print(assistant_response)

Slide 1: 프로젝트 개요
Content:
- 프로젝트 목적
- 프로젝트 범위
- 프로젝트 목표

Slide 2: 현황 분석
Content:
- 현재 데이터 관리 시스템 분석
- 문제점 파악
- 개선이 필요한 부분

Slide 3: 요구사항 분석
Content:
- 고객 요구사항 분석
- 시스템 요구사항
- 기능 요구사항

Slide 4: 아키텍처 설계
Content:
- 데이터베이스 구조
- 시스템 구성도
- 통합 플랫폼 아키텍처

Slide 5: 기술적 해결방안
Content:
- 데이터 통합 방법
- 보안 및 안정성 강화 방안
- 성능 최적화 방안

Slide 6: 일정 계획
Content:
- 프로젝트 일정
- 주요 마일스톤
- 작업 일정

Slide 7: 예산 산정
Content:
- 프로젝트 총 비용
- 예산 산정 방법
- 비용 분배 계획

Slide 8: 이슈 및 위험관리
Content:
- 프로젝트 진행 중 발생할 수 있는 이슈
- 위험 분석
- 대응 전략

Slide 9: 협업 및 의사소통 계획
Content:
- 프로젝트 팀 구성
- 의사소통 도구 활용 계획
- 회의 일정

Slide 10: 추진 전략
Content:
- 프로젝트 추진 방안
- 성과 평가 기준
- 지속적인 개선 방안

EndOfDocument


In [21]:
slides_outline = parse_content(assistant_response)
print(slides_outline)

[{'title': '프로젝트 개요', 'content': [{'프로젝트 목적': '프로젝트 목적'}, {'프로젝트 범위': '프로젝트 범위'}, {'프로젝트 목표': '프로젝트 목표'}]}, {'title': '현황 분석', 'content': [{'현재 데이터 관리 시스템 분석': '현재 데이터 관리 시스템 분석'}, {'문제점 파악': '문제점 파악'}, {'개선이 필요한 부분': '개선이 필요한 부분'}]}, {'title': '요구사항 분석', 'content': [{'고객 요구사항 분석': '고객 요구사항 분석'}, {'시스템 요구사항': '시스템 요구사항'}, {'기능 요구사항': '기능 요구사항'}]}, {'title': '아키텍처 설계', 'content': [{'데이터베이스 구조': '데이터베이스 구조'}, {'시스템 구성도': '시스템 구성도'}, {'통합 플랫폼 아키텍처': '통합 플랫폼 아키텍처'}]}, {'title': '기술적 해결방안', 'content': [{'데이터 통합 방법': '데이터 통합 방법'}, {'보안 및 안정성 강화 방안': '보안 및 안정성 강화 방안'}, {'성능 최적화 방안': '성능 최적화 방안'}]}, {'title': '일정 계획', 'content': [{'프로젝트 일정': '프로젝트 일정'}, {'주요 마일스톤': '주요 마일스톤'}, {'작업 일정': '작업 일정'}]}, {'title': '예산 산정', 'content': [{'프로젝트 총 비용': '프로젝트 총 비용'}, {'예산 산정 방법': '예산 산정 방법'}, {'비용 분배 계획': '비용 분배 계획'}]}, {'title': '이슈 및 위험관리', 'content': [{'프로젝트 진행 중 발생할 수 있는 이슈': '프로젝트 진행 중 발생할 수 있는 이슈'}, {'위험 분석': '위험 분석'}, {'대응 전략': '대응 전략'}]}, {'title': '협업 및 의사소통 계획', 'content': [{'프로젝트 팀 구성': '프로젝트 

In [22]:
slides_content = make_content(slides_outline)
print(slides_content)

Make Content : 프로젝트 개요 - 프로젝트 목적 -{'프로젝트 목적': '프로젝트 목적은 기존 데이터 관리 시스템의 문제점을 개선하고, 효율적인 데이터 관리 및 시스템 운영을 위한 새로운 아키텍처를 설계하고 구축하는 것이다. 이를 통해 기업의 데이터 활용성과 보안성을 향상시키고, 비즈니스 프로세스를 최적화하여 기업의 경쟁력을 강화하는 것이 목표이다.'}
Make Content : 프로젝트 개요 - 프로젝트 범위 -{'프로젝트 범위': '프로젝트 범위는 새로운 데이터 관리 시스템을 구축하고, 기존 시스템의 문제를 개선하여 데이터 통합성, 보안성, 성능을 향상시키는 것을 목표로 합니다. 이에 따라 데이터베이스 구조를 재설계하고, 시스템 구성을 개선하며, 통합 플랫폼 아키텍처를 구축하는 것이 프로젝트 범위에 포함됩니다.'}
Make Content : 프로젝트 개요 - 프로젝트 목표 -{'프로젝트 목표': '프로젝트 목표:\n- 고객 데이터 관리 시스템을 효율적으로 구축하여 데이터 접근성과 보안성을 향상시키고, 실시간 데이터 처리 및 분석을 가능케 하여 업무 프로세스를 최적화한다. 데이터의 정확성과 신속한 처리를 보장하여 고객 서비스 품질을 향상시키는 것을 목표로 한다.'}
Make Content : 현황 분석 - 현재 데이터 관리 시스템 분석 -{'현재 데이터 관리 시스템 분석': '현재 데이터 관리 시스템은 분산된 데이터베이스를 사용하며 업무 프로세스 간의 연계가 부족합니다. 이로 인해 실시간 데이터 공유가 어려우며 중복된 데이터 발생이 잦습니다. 또한 보안 문제로 인해 데이터 무결성을 보장하기 어렵고, 확장성과 성능에 제약이 있습니다. 이에 개선이 필요한 부분은 데이터 일원화, 보안 강화, 성능 최적화, 데이터 품질 향상 등이 있습니다.'}
Make Content : 현황 분석 - 문제점 파악 -{'문제점 파악': '[문제점 파악]\n현황 분석을 통해 데이터 관리 시스템의 확장성 부족, 데이터 무결성 문제, 사용자 경험 부족 등의 문제가 드러났습니다.

In [23]:
dir_path = "./"
template_path = os.path.join(dir_path, "ppt_template.pptx")
prs = Presentation(template_path)

# 표지 : 0 Title 1, 13 Picture Placeholder 2, 14 Text Placeholder 3
slide = prs.slides.add_slide(prs.slide_layouts[0])
slide.placeholders[13].insert_picture("title.jpg")
slide.placeholders[0].text = presentation_title
slide.placeholders[14].text = presenter_name

# 목차 : 0 Title 1, 1 Content Placeholder 2, 13 Picture Placeholder 3, 14 Picture Placeholder 4, 15 Picture Placeholder 5
slide = prs.slides.add_slide(prs.slide_layouts[1])
slide.placeholders[0].text = 'Content'
slide.placeholders[1].text = """1. 기획
2. 설계
3. 인사이트
"""
slide.placeholders[13].insert_picture("index1.jpg")
slide.placeholders[14].insert_picture("index2.jpg")
slide.placeholders[15].insert_picture("index3.jpg")

# 챕터 장표
slide = prs.slides.add_slide(prs.slide_layouts[4])
slide.placeholders[0].text = "1. 기획"
slide.placeholders[13].insert_picture("subject.jpg")


for s in slides_content:
    count = len(s['content'])
    if count == 1:
        k = list(s['content'][0].keys())
        slide = prs.slides.add_slide(prs.slide_layouts[5])
        slide.placeholders[0].text = s['title']
        slide.placeholders[1].text = s['content'][0][k[0]]
    elif count == 2:
        k1 = list(s['content'][0].keys())
        k2 = list(s['content'][1].keys())
        slide = prs.slides.add_slide(prs.slide_layouts[8])
        slide.placeholders[0].text = s['title']
        slide.placeholders[1].text = k1[0]
        slide.placeholders[2].text = s['content'][0][k1[0]]
        slide.placeholders[3].text = k2[0]
        slide.placeholders[4].text = s['content'][1][k2[0]]
    else:
        k1 = list(s['content'][0].keys())
        k2 = list(s['content'][1].keys())
        k3 = list(s['content'][2].keys())
        slide = prs.slides.add_slide(prs.slide_layouts[9])
        slide.placeholders[0].text = s['title']
        slide.placeholders[1].text = k1[0]
        slide.placeholders[2].text = s['content'][0][k1[0]]
        slide.placeholders[3].text = k2[0]
        slide.placeholders[4].text = s['content'][1][k2[0]]
        slide.placeholders[13].text= k3[0]
        slide.placeholders[14].text= s['content'][2][k3[0]]

# 마지막장
slide = prs.slides.add_slide(prs.slide_layouts[10])
slide.placeholders[0].text = "감사합니다."
slide.placeholders[13].insert_picture("ending.jpg")
slide.placeholders[15].text = """본 자료는 생성형 AI로 만들었습니다. 
내용을 확인하고 수정하여 사용하세요."""

# PPT 템플릿 슬라이드 삭제
for slide_id in [7, 6, 5, 4, 3, 2, 1, 0]:
    if slide_id < len(prs.slides):
        xml_slides = prs.slides._sldIdLst
        slides = list(xml_slides)
        xml_slides.remove(slides[slide_id])


# Save the presentation
prs.save(os.path.join(dir_path, 'generated_ppt.pptx'))

FileNotFoundError: [Errno 2] No such file or directory: 'title.jpg'

# 파워포인트 레이아웃 확인

In [ ]:
# 기본 레이아웃 11가지 placeholder 출력
template_path = os.path.join(dir_path, "ppt_template.pptx")
prs = Presentation(template_path)

for i in range(0, 11):
    print(f"\n======={i+1}번 슬라이드=======")
    slide = prs.slides.add_slide(prs.slide_layouts[i])
    
    # 생성한 레이아웃 안의 모든 placeholder 이름 확인
    for shape in slide.placeholders:
        #print(f"{shape.name}")
        print(f"{shape.placeholder_format.idx} {shape.name}")